In [1]:
from libra.retrievers import VectorRetriever
from libra.vectordb import MilvusStorage
from libra.embeddings import SentenceTransformerEncoder, OpenAIEmbedding

import os
from dotenv import load_dotenv

load_dotenv(override=True)

embedding_instance = SentenceTransformerEncoder()

/home/gcp_dpf/libra-core/.venv/lib/python3.9/site-packages/sentence_transformers/cross_encoder/CrossEncoder.py:11: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm, trange


In [2]:
mb_info_storage_instance = MilvusStorage(
    vector_dim=embedding_instance.get_output_dim(),
    url_and_api_key=(os.environ['MILVUS_URI'], "123123"),
    collection_name="MBInfo"
)
# mb_info_storage_instance.clear()

In [3]:
mb_info_retriever = VectorRetriever(
    embedding_model=embedding_instance, 
    storage=mb_info_storage_instance
)

### Adding Company Profile

In [4]:
from libra.loaders import read_file
from io import BytesIO
import json
from libra.utils import json_to_well_formatted

company_profiles = []


file_path = os.path.join('..', '..', 'data', 'company_profile.csv')
with open(file_path, "rb") as f:
    file = BytesIO(f.read())
    file.name = "company_profile.csv"
    json_file = read_file(file)
    for doc in json_file.docs:
        doc_content = json.loads(doc['page_content'])    
        company_profiles.append({
            "embed": doc_content['Documents Topic'],
            "content": json_to_well_formatted({
                "Topic": doc_content['Documents Topic'],
                "Content": doc_content['Document Content'],
            })
        })

In [35]:
mb_info_retriever.process(data=company_profiles, batch_size=1)

100%|██████████| 169/169 [00:26<00:00,  6.48it/s]


In [9]:
results = mb_info_retriever.query(query="Tổng giám đốc đã nói gì ?", top_k=4)

for entry in results:
    print(entry['metadata'])

{'embed': 'Thông điệp của Tổng Giám đốc MB'}
{'embed': 'Thông điệp của Chủ tịch Hội đồng Quản trị MB'}
{'embed': 'Báo Cáo Đánh Giá Của Hội Đồng Quản Trị'}
{'embed': 'Thành viên ban điều hành'}


### Adding jobs

In [7]:
job_storage_instance = MilvusStorage(
    vector_dim=embedding_instance.get_output_dim(),
    url_and_api_key=(os.environ['MILVUS_URI'], "123123"),
    collection_name="jobs"
)
# job_storage_instance.clear()

In [8]:
job_retriever = VectorRetriever(
    embedding_model=embedding_instance,
    storage=job_storage_instance
)

In [51]:
import re

def parse_salaries(salary_data):
    salaries = ("", "")
    lines = salary_data.split('\n')
    
    for line in lines:
        line = line.strip()
        if not line:
            continue
        
        # Check for "a - b VND" format
        match = re.match(r'(\d{1,3}(?:,\d{3})*) - (\d{1,3}(?:,\d{3})*)\s*VNĐ', line)
        if match:
            min_salary = match.group(1).replace(',', '')
            max_salary = match.group(2).replace(',', '')
        else:
            # For other formats, set min and max to the same value
            # Remove "VNĐ" and any leading/trailing whitespace
            value = line.replace('VNĐ', '').strip()
            
            if value in ['Lương thỏa thuận', 'Cạnh Tranh']:
                min_salary = max_salary = value
            else:
                # Try to extract a number if present
                number_match = re.search(r'\d{1,3}(?:,\d{3})*', value)
                if number_match:
                    salary_value = int(number_match.group().replace(',', ''))
                    min_salary = max_salary = salary_value
                else:
                    min_salary = max_salary = value

        salaries = (min_salary, max_salary)
    
    return salaries

def is_number(s):
    return s.isdigit()

In [52]:
from libra.loaders import read_file
from io import BytesIO
import json
from pprint import pprint

jobs = []

keys_to_extract = ["title", "workplace", "industry", "welfare", "min_salary", "max_salary"]

file_path = os.path.join('..', '..', 'data', 'jobs_data.json')
with open(file_path, "rb") as f:
    file = BytesIO(f.read())
    file.name = "jobs_data.json"
    json_file = read_file(file)
    for doc in json_file.docs:
        doc_content = json.loads(doc['page_content']) 
        a, b = parse_salaries(doc_content['salary'])
        if is_number(a):
            a +=' VNĐ' # type: ignore
            
        if is_number(b):
            b +=' VNĐ' # type: ignore
        
        doc_content['min_salary'] = a
        doc_content['max_salary'] = b
            
        job = {
            "embed": str({key: doc_content[key] for key in keys_to_extract}),
            "content": str(doc_content)
        }
        jobs.append(job)

In [53]:
jobs[-1]['embed']

"{'title': 'Chuyên viên/CVCC Quản trị rủi ro mô hình - Khối Quản trị rủi ro', 'workplace': 'Hà Nội', 'industry': 'Ngân hàng, Tài chính / Đầu tư', 'welfare': ['Bảo hiểm', 'Du Lịch', 'Thưởng', 'Chăm sóc sức khỏe', 'Đào tạo', 'Tăng lương'], 'min_salary': 'Lương thỏa thuận', 'max_salary': 'Lương thỏa thuận'}"

In [54]:
job_storage_instance.clear()

In [55]:
job_retriever.process(jobs, batch_size=1)

100%|██████████| 365/365 [03:05<00:00,  1.97it/s]


In [11]:
for entry in job_retriever.query("nghề UB ở Hải Phòng", top_k=5):
    print(entry['metadata']['embed'])

{'title': 'Chuyên viên UB (Chuyên viên Tư vấn) - CN. Nam Hải Phòng', 'workplace': 'Hải Phòng', 'industry': 'Dịch vụ khách hàng, Ngân hàng, Tài chính / Đầu tư', 'welfare': ['Bảo hiểm', 'Du Lịch', 'Phụ cấp', 'Du lịch nước ngoài', 'Đồng phục', 'Thưởng', 'Chăm sóc sức khỏe', 'Đào tạo', 'Tăng lương', 'Công tác phí', 'Phụ cấp thâm niên', 'Chế độ nghỉ phép', 'Chế độ tập thể dục'], 'min_salary': '8000000 VNĐ', 'max_salary': '15000000 VNĐ'}
{'title': 'CN Hậu Giang - Chuyên viên UB', 'workplace': 'Hậu Giang', 'industry': 'Bán hàng / Kinh doanh, Ngân hàng, Tài chính / Đầu tư', 'welfare': ['Bảo hiểm', 'Du Lịch', 'Phụ cấp', 'Du lịch nước ngoài', 'Đồng phục', 'Thưởng', 'Chăm sóc sức khỏe', 'Đào tạo', 'Tăng lương', 'Công tác phí', 'Phụ cấp thâm niên', 'Chế độ nghỉ phép', 'Chế độ tập thể dục'], 'min_salary': '10000000 VNĐ', 'max_salary': '35000000 VNĐ'}
{'title': 'Chuyên viên UB (Chuyên viên Tư vấn) - CN. Nghi Sơn (Thanh Hóa)', 'workplace': 'Thanh Hóa', 'industry': 'Dịch vụ khách hàng, Ngân hàng, Tài 